In [143]:
# Imports
import plotly.express as px
import panel as pn
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv
import yfinance as yf
import requests
import hvplot.pandas
import json
from MCForecastTools import MCSimulation
import seaborn as sns

%matplotlib inline


In [144]:
# Setup all DataFrames
# Gold

gold = yf.download('GC=F', period='max', interval = "1mo")
gold_data = pd.DataFrame(gold['Close']).dropna().rename(columns={'Close':'close'})

# Oil

oil = yf.download("CL=F", period='max', interval = "1mo")
oil_data = pd.DataFrame(oil['Close']).dropna().rename(columns={'Close':'close'})

# S&P 500

snp = yf.download('^GSPC', period='max', interval = "1mo")
snp_data = pd.DataFrame(snp['Close']).dropna().rename(columns={'Close':'close'})

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [145]:
today = pd.to_datetime('today')
print(today)

2022-04-07 20:35:45.600758


In [146]:
# Slice out Data
# GFC

gold_data_gfc = gold_data.loc["2006-06-01":"2010-01-01"]
oil_data_gfc = oil_data.loc["2006-06-01":"2010-01-01"]
snp_data_gfc = snp_data.loc["2006-06-01":"2010-01-01"]

# 9/11 and Afghan

gold_data_911 = gold_data.loc["2000-10-01":"2005-08-01"]
oil_data_911 = oil_data.loc["2000-10-01":"2005-08-01"]
snp_data_911 = snp_data.loc["2000-10-01":"2005-08-01"]

# COVID-19

gold_data_covid = gold_data.loc["2018-12-31":today]
oil_data_covid = oil_data.loc["2018-12-31":today]
snp_data_covid = snp_data.loc["2018-12-31":today]

# Potential long-term effects of war from the analysis of previous war data? Ukraine Russia War
# Afghanistan war

gold_data_afghan = gold_data.loc["2015-10-01":"2021-08-01"]
oil_data_afghan = oil_data.loc["2015-10-01":"2021-08-01"]
snp_data_afghan = snp_data.loc["2015-10-01":"2021-08-01"]

# Iraq war

gold_data_iraq = gold_data.loc["2007-05-01":"2012-12-01"]
oil_data_iraq = oil_data.loc["2007-05-01":"2012-12-01"]
snp_data_iraq = snp_data.loc["2007-05-01":"2012-12-01"]

# What are the current short-term effects of war? (Ukraine Russia War)

gold_data_short = gold_data.loc["2022-02-24":today]
oil_data_short = oil_data.loc["2022-02-24":today]
snp_data_short = snp_data.loc["2022-02-24":today]


In [147]:
gold_data_afghan.head()

,close
Date,
2015-10-01,1141.500000
2015-12-01,1060.300049
2016-01-01,1116.400024
2016-02-01,1233.900024
2016-03-01,1234.199951


In [148]:
# Concat by every event
# GFC
event_gfc = pd.concat(
    [gold_data_gfc, oil_data_gfc, snp_data_gfc], axis="columns", join="inner"
)
# 9/11

event_911 = pd.concat(
    [gold_data_911, oil_data_911, snp_data_911], axis="columns", join="inner"
)

# COVID-19

event_covid = pd.concat(
    [gold_data_covid, oil_data_covid, snp_data_covid], axis="columns", join="inner"
)

# Potential long-term effects of war from the analysis of previous war data? Ukraine Russia War

# What are the current short-term effects of war? (Ukraine Russia War)

event_short = pd.concat(
    [gold_data_short, oil_data_short, snp_data_short], axis="columns", join="inner"
)

In [149]:
# Slice event_long data

gold_data_afghan['symbol'] = 'GC=F'
oil_data_afghan['symbol'] = 'CL=F'
snp_data_afghan['symbol'] = '^GSPC'

gold_data_iraq['symbol'] = 'GC=F'
oil_data_iraq['symbol'] = 'CL=F'
snp_data_iraq['symbol'] = '^GSPC'

# Format Data for Gold Afghan
def format_MCSimulation(gold_data_afghan):
    ticker_list = gold_data_afghan['symbol'].unique()
    df_ = pd.DataFrame()
    for ticker in ticker_list:
        df_ticker_reformatted_gold_afghan = gold_data_afghan[gold_data_afghan['symbol'] == ticker]
        df_ticker_reformatted_gold_afghan.columns = pd.MultiIndex.from_product([[ticker],gold_data_afghan.columns])
        if df_.empty:
            df_ = df_ticker_reformatted_gold_afghan
        else:
            df_ = pd.concat([df_,
                            df_ticker_reformatted_gold_afghan],
                            axis = 1,
                            join='inner')

    return df_
df_ticker_reformatted_gold_afghan = format_MCSimulation(gold_data_afghan)

# Format Data for Oil Afghan
def format_MCSimulation(oil_data_afghan):
    ticker_list = oil_data_afghan['symbol'].unique()
    df_ = pd.DataFrame()
    for ticker in ticker_list:
        df_ticker_reformatted_oil_afghan = oil_data_afghan[oil_data_afghan['symbol'] == ticker]
        df_ticker_reformatted_oil_afghan.columns = pd.MultiIndex.from_product([[ticker],oil_data_afghan.columns])
        if df_.empty:
            df_ = df_ticker_reformatted_oil_afghan
        else:
            df_ = pd.concat([df_,
                            df_ticker_reformatted_oil_afghan],
                            axis = 1,
                            join='inner')

    return df_
df_ticker_reformatted_oil_afghan = format_MCSimulation(oil_data_afghan)

# Format Data for S&P 500 Afghan
def format_MCSimulation(snp_data_afghan):
    ticker_list = snp_data_afghan['symbol'].unique()
    df_ = pd.DataFrame()
    for ticker in ticker_list:
        df_ticker_reformatted_snp_afghan = snp_data_afghan[snp_data_afghan['symbol'] == ticker]
        df_ticker_reformatted_snp_afghan.columns = pd.MultiIndex.from_product([[ticker],snp_data_afghan.columns])
        if df_.empty:
            df_ = df_ticker_reformatted_snp_afghan
        else:
            df_ = pd.concat([df_,
                            df_ticker_reformatted_snp_afghan],
                            axis = 1,
                            join='inner')

    return df_
df_ticker_reformatted_snp_afghan = format_MCSimulation(snp_data_afghan)

# Format Data for Gold Iraq

def format_MCSimulation(gold_data_iraq):
    ticker_list = gold_data_iraq['symbol'].unique()
    df_ = pd.DataFrame()
    for ticker in ticker_list:
        df_ticker_reformatted_gold_iraq = gold_data_iraq[gold_data_iraq['symbol'] == ticker]
        df_ticker_reformatted_gold_iraq.columns = pd.MultiIndex.from_product([[ticker],gold_data_iraq.columns])
        if df_.empty:
            df_ = df_ticker_reformatted_gold_iraq
        else:
            df_ = pd.concat([df_,
                            df_ticker_reformatted_gold_iraq],
                            axis = 1,
                            join='inner')

    return df_
df_ticker_reformatted_gold_iraq = format_MCSimulation(gold_data_iraq)

# Format Data for Oil Iraq

def format_MCSimulation(oil_data_iraq):
    ticker_list = oil_data_iraq['symbol'].unique()
    df_ = pd.DataFrame()
    for ticker in ticker_list:
        df_ticker_reformatted_oil_iraq = oil_data_iraq[oil_data_iraq['symbol'] == ticker]
        df_ticker_reformatted_oil_iraq.columns = pd.MultiIndex.from_product([[ticker],oil_data_iraq.columns])
        if df_.empty:
            df_ = df_ticker_reformatted_oil_iraq
        else:
            df_ = pd.concat([df_,
                            df_ticker_reformatted_oil_iraq],
                            axis = 1,
                            join='inner')

    return df_
df_ticker_reformatted_oil_iraq = format_MCSimulation(oil_data_iraq)

# Format Data for S&P 500 Iraq

def format_MCSimulation(snp_data_iraq):
    ticker_list = snp_data_iraq['symbol'].unique()
    df_ = pd.DataFrame()
    for ticker in ticker_list:
        df_ticker_reformatted_snp_iraq = snp_data_iraq[snp_data_iraq['symbol'] == ticker]
        df_ticker_reformatted_snp_iraq.columns = pd.MultiIndex.from_product([[ticker],snp_data_iraq.columns])
        if df_.empty:
            df_ = df_ticker_reformatted_snp_iraq
        else:
            df_ = pd.concat([df_,
                            df_ticker_reformatted_snp_iraq],
                            axis = 1,
                            join='inner')

    return df_
df_ticker_reformatted_snp_iraq = format_MCSimulation(snp_data_iraq)

C:\Users\alexl\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\alexl\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\alexl\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on 

In [150]:
# Monte Carlo Simulation for potential long-term effects of war from the analysis of previous war data
# Gold Afghan

monte_carlo_gold_afghan = MCSimulation(
    portfolio_data = df_ticker_reformatted_gold_afghan,
    num_simulation = 500,
    num_trading_days = 60
)

# Oil Afghan

monte_carlo_oil_afghan = MCSimulation(
    portfolio_data = df_ticker_reformatted_oil_afghan,
    num_simulation = 500,
    num_trading_days = 60
)

# S&P 500 Afghan

monte_carlo_snp_afghan = MCSimulation(
    portfolio_data = df_ticker_reformatted_snp_afghan,
    num_simulation = 500,
    num_trading_days = 60
)

# Gold Iraq

monte_carlo_gold_iraq = MCSimulation(
    portfolio_data = df_ticker_reformatted_gold_iraq,
    num_simulation = 500,
    num_trading_days = 60
)

# Oil Iraq

monte_carlo_oil_iraq = MCSimulation(
    portfolio_data = df_ticker_reformatted_oil_iraq,
    num_simulation = 500,
    num_trading_days = 60
)

# S&P 500 Iraq

monte_carlo_snp_iraq = MCSimulation(
    portfolio_data = df_ticker_reformatted_snp_iraq,
    num_simulation = 500,
    num_trading_days = 60
)


In [151]:
# Running Monte Carlo Simulations
# Gold Afghan
monte_carlo_gold_afghan.calc_cumulative_return()
# Oil Afghan
monte_carlo_oil_afghan.calc_cumulative_return()
# S&P500 Afghan
monte_carlo_snp_afghan.calc_cumulative_return()
# Gold Iraq
monte_carlo_gold_iraq.calc_cumulative_return()
# Oil Iraq
monte_carlo_oil_iraq.calc_cumulative_return()
# S&P500 Iraq
monte_carlo_snp_iraq.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.


C:\Users\alexl\Documents\Monash_Fintech\Project1_group1\MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Monte Carlo simulation number 360.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.977909,0.998951,0.997592,0.923708,1.045688,0.980958,0.903090,1.046636,0.943317,0.968872,...,1.013556,1.056592,0.985259,0.971284,0.953281,0.961107,1.034978,1.069032,1.041974,0.946757
2,0.852489,0.998406,1.063411,0.930464,1.074207,0.984059,0.877027,0.979850,1.026113,0.980958,...,1.055052,1.114834,0.978503,1.001742,0.938081,0.919874,1.098578,1.135368,1.033718,0.915902
3,0.866451,1.075959,1.075514,0.945461,1.153377,0.954042,0.832058,1.025584,0.978705,0.962347,...,1.013232,1.034684,1.013384,0.949001,0.871809,0.910177,1.026572,1.190452,1.045356,0.863577
4,0.912892,0.941085,0.994871,0.989101,1.141284,1.017194,0.821658,0.977658,1.092427,0.936998,...,1.058909,1.013580,1.053026,0.957375,0.853772,0.970119,0.954633,1.181005,1.072093,0.872623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,1.425813,0.598277,1.336612,1.611861,1.227752,0.833096,0.938527,0.939841,0.663767,0.843747,...,0.668150,1.010024,0.745676,0.669435,1.100955,1.278281,2.489899,0.741918,1.996031,0.646994
57,1.335335,0.553503,1.311977,1.695284,1.139094,0.845976,0.913650,0.941932,0.678964,0.800848,...,0.653284,1.077706,0.774355,0.688038,1.120651,1.179489,2.440701,0.727658,2.088183,0.691332
58,1.218771,0.547513,1.304802,1.784853,1.159776,0.797855,0.881858,0.996553,0.729130,0.813951,...,0.665114,1.059380,0.820221,0.657361,1.037248,1.146855,2.463731,0.740148,2.111641,0.646355
59,1.341116,0.566820,1.232768,1.823330,1.147323,0.810374,0.816953,0.929242,0.709343,0.871007,...,0.699485,1.076789,0.761559,0.652433,1.005695,1.162605,2.418840,0.756143,2.004141,0.610028


In [178]:
# Compute summary statistics from the simulated monthly returns
simulated_gold_afghan_returns_data = {
    "mean": list(monte_carlo_gold_afghan.simulated_return.mean(axis='columns')),
    "median": list(monte_carlo_gold_afghan.simulated_return.median(axis='columns')),
    "min": list(monte_carlo_gold_afghan.simulated_return.min(axis='columns')),
    "max": list(monte_carlo_gold_afghan.simulated_return.max(axis='columns'))
}


simulated_oil_afghan_returns_data = {
    "mean": list(monte_carlo_oil_afghan.simulated_return.mean(axis='columns')),
    "median": list(monte_carlo_oil_afghan.simulated_return.median(axis='columns')),
    "min": list(monte_carlo_oil_afghan.simulated_return.min(axis='columns')),
    "max": list(monte_carlo_oil_afghan.simulated_return.max(axis='columns'))
}

simulated_snp_afghan_returns_data = {
    "mean": list(monte_carlo_snp_afghan.simulated_return.mean(axis='columns')),
    "median": list(monte_carlo_snp_afghan.simulated_return.median(axis='columns')),
    "min": list(monte_carlo_snp_afghan.simulated_return.min(axis='columns')),
    "max": list(monte_carlo_snp_afghan.simulated_return.max(axis='columns'))
}

simulated_gold_iraq_returns_data = {
    "mean": list(monte_carlo_gold_iraq.simulated_return.mean(axis='columns')),
    "median": list(monte_carlo_gold_iraq.simulated_return.median(axis='columns')),
    "min": list(monte_carlo_gold_iraq.simulated_return.min(axis='columns')),
    "max": list(monte_carlo_gold_iraq.simulated_return.max(axis='columns'))
}

simulated_oil_iraq_returns_data = {
    "mean": list(monte_carlo_oil_iraq.simulated_return.mean(axis='columns')),
    "median": list(monte_carlo_oil_iraq.simulated_return.median(axis='columns')),
    "min": list(monte_carlo_oil_iraq.simulated_return.min(axis='columns')),
    "max": list(monte_carlo_oil_iraq.simulated_return.max(axis='columns'))
}

simulated_snp_iraq_returns_data = {
    "mean": list(monte_carlo_snp_iraq.simulated_return.mean(axis='columns')),
    "median": list(monte_carlo_snp_iraq.simulated_return.median(axis='columns')),
    "min": list(monte_carlo_snp_iraq.simulated_return.min(axis='columns')),
    "max": list(monte_carlo_snp_iraq.simulated_return.max(axis='columns'))
}

# Create a DataFrames with the summary statistics

df_simulated_returns_gold_afghan = pd.DataFrame(simulated_gold_afghan_returns_data).rename(columns={'mean':'Gold After Afghan War'})

df_simulated_returns_oil_afghan = pd.DataFrame(simulated_oil_afghan_returns_data).rename(columns={'mean':'Oil After Afghan War'})

df_simulated_returns_snp_afghan = pd.DataFrame(simulated_snp_afghan_returns_data).rename(columns={'mean':'S&P 500 After Afghan War'})
    
df_simulated_returns_gold_iraq = pd.DataFrame(simulated_gold_iraq_returns_data).rename(columns={'mean':'Gold After Iraq War'})

df_simulated_returns_oil_iraq = pd.DataFrame(simulated_oil_iraq_returns_data).rename(columns={'mean':'Oil After Iraq War'})

df_simulated_returns_snp_iraq = pd.DataFrame(simulated_snp_iraq_returns_data).rename(columns={'mean':'S&P 500 After Iraq War'})

# Configure data
# Gold after Afghan

plot_gold_afghan = df_simulated_returns_gold_afghan[['Gold After Afghan War']]

# Oil after Afghan

plot_oil_afghan = df_simulated_returns_oil_afghan[['Oil After Afghan War']]

# S&P 500 after Afghan

plot_snp_afghan = df_simulated_returns_snp_afghan[['S&P 500 After Afghan War']]

# Gold after Iraq

plot_gold_iraq = df_simulated_returns_gold_iraq[['Gold After Iraq War']]

# Oil after Iraq

plot_oil_iraq = df_simulated_returns_oil_iraq[['Oil After Iraq War']]

# S&P 500 after Iraq

plot_snp_iraq = df_simulated_returns_snp_iraq[['S&P 500 After Iraq War']]


In [179]:
combined_df_iraq = pd.concat([plot_gold_iraq, plot_oil_iraq, plot_snp_iraq])

In [195]:
combined_df_iraq.hvplot(label="Monte Carlo Simulation of Gold, Oil, and S&P 500 after the Iraq War (60 Months = 5 years)", width = 800)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [191]:
combined_df_afghan = pd.concat([plot_gold_afghan, plot_oil_afghan, plot_snp_afghan])

In [192]:
combined_df_afghan.hvplot(label="Monte Carlo Simulation of Gold, Oil, and S&P 500 after the Afghanistan War (60 Months = 5 years)", width = 800)

:NdOverlay   [Variable]
   :Curve   [index]   (value)